In [ ]:
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
from pathlib import Path
import re
import xarray as xr

import metpy.calc as mcalc
import metpy.units as metunits
# local module
import mypaths

import json

from ipywidgets import interact
from tqdm import tqdm_notebook as tqdm

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
plt.rcParams['figure.figsize'] = (12, 7)

In [ ]:
from parse_logs import AllianceComposite, MSG_LIST, average_ds_over_time

In [ ]:
with open('weatherpack_variable_aliases.json', 'r') as fj:
    vrbl_aliases = json.load(fj)
vrbl_aliases

In [ ]:
wpk_usage = pd.read_csv('weatherpack_usage.txt',
                        sep='\s+',
                        na_values='NA',
                        parse_dates=['date'],
                        index_col='date',
#                         dtype=dict(wpk2=str),
                        ).fillna('')
wpk_usage.head()

In [ ]:
vrbl_attrs = {
    't': {
        'name': 'air_temperature',
        'attrs': {
            'standard_name': 'air_temperature',
            'long_name': 'Air Temperature',
            'units': 'degree_celsius'
        }
    },
    'p': {
        'scale': 100,
        'name': 'air_pressure',
        'attrs': {
            'standard_name': 'air_pressure',
            'long_name': 'Air Pressure',
            'units': 'Pa'
        }
    },
    'rh': {
        'scale': 1e-2,
        'name': 'relative_humidity',
        'attrs': {
            'standard_name': 'relative_humidity',
            'long_name': 'Relative Humidity',
            'units': '1'
        }
    },
    'sr': {
        'name': 'solar_irradiance',
        'attrs': {
            'standard_name': 'solar_irradiance',
            'long_name': 'Solar Irradiance',
            'units': 'W m-2'
        }
    },
    'u': {
        'name': 'u',
        'attrs': {
            'standard_name': 'eastward_wind',
            'long_name': 'U component of wind',
            'units': 'm s-1'
        },
        'dep': ['wd', 'ws']
    },
    'v': {
        'name': 'v',
        'attrs': {
            'standard_name': 'northward_wind',
            'long_name': 'V component of wind',
            'units': 'm s-1'
        },
        'dep': ['wd', 'ws']
    }
}

In [ ]:
def wpk_latlon_parser(s):
    latlon_re = re.compile(r'''
# Latitude part
(?P<lat_hem>[NS])\s*
(?P<lat_deg>[0-9]{2})\s*
(?P<lat_min>[0-9]{1,2}\.[0-9]{3})\s*

# Longitude part
(?P<lon_hem>[EW])\s*
(?P<lon_deg>[0-9]{3})\s*
(?P<lon_min>[0-9]{1,2}\.[0-9]{3})''', re.X)
    
    m = re.match(latlon_re, s)
    if m:
        if m.group('lat_hem') == 'S':
            lat_factor = -1
        else:
            lat_factor = 1

        if m.group('lon_hem') == 'W':
            lon_factor = -1
        else:
            lon_factor = 1

        lat = lat_factor * (float(m.group('lat_deg')) + float(m.group('lat_min')) / 60)
        lon = lon_factor * (float(m.group('lon_deg')) + float(m.group('lon_min')) / 60)
    lat, lon = np.nan, np.nan
    
    return lat, lon

In [ ]:
# time_range = (pd.date_range(start=date,
#                             freq='T',
#                             end=date+timedelta(hours=23, minutes=59, seconds=59))
#               .to_series()
#               .to_frame(name='time'))

In [ ]:
# inputdir = mypaths.wpk_dir / '2_Leg' / 'TRUEWIND'

In [ ]:
# fname = inputdir / f'Wpk_st04@{date:%Y_%m_%d}.txt'
# print(fname.exists())
# # fname = inputdir / f'data_3_{date:%Y%m%d_%H}.log'

In [ ]:
def interp_dataframe_time(df, date, freq='1T', end='auto'):
    if end == 'auto':
        end = date + timedelta(hours=23, minutes=59, seconds=59)
    time_range = (pd.date_range(start=date,
                                freq=freq,
                                end=end)
                      .to_series()
                      .to_frame(name='time'))

    labels = time_range.index
    df = (pd.concat([df, time_range])
          .sort_index()
          .interpolate(method='values', limit=1)
          .drop('time', axis=1))
    df.index = df.index.rename('time')
    df = df.loc[df.index.intersection(labels)]
    return df[~df.index.duplicated(keep='first')]

In [ ]:
def add_wind_components(df):
    wspd = None
    wdir = None
    for alias in vrbl_aliases['ws']:
        try:
            wspd = df[alias]
        except KeyError:
            pass
    for alias in vrbl_aliases['wd']:
        try:
            wdir = df[alias]
        except KeyError:
            pass

    if wspd is not None and wdir is not None:
        df['u'], df['v'] = mcalc.get_wind_components(wspd.values * metunits.units('m/s'),
                                                     wdir.values * metunits.units('degrees'))
    return df

In [ ]:
def read_wpk_daily(topdir, date, wpk_id):
    wpk_id = str(wpk_id)
    assert wpk_id in ['2', '4'], 'Works only for WeatherPacks  No. 2 or 4'
    
    df = pd.DataFrame()
    if wpk_id == '2':
        fname = topdir / f'Wpk_st0{wpk_id}@{date:%Y_%m_%d}.txt'
        if fname.exists():
            df = pd.read_csv(fname, parse_dates=[[1, 2]], index_col=0,
                             date_parser=lambda x: datetime.strptime(x, '%y/%m/%d %H:%M:%S'))
            # df.index.rename('DateTime', inplace=True)
            df[['latitude', 'longitude']] = (df['Ship position']
                                             .map(wpk_latlon_parser, na_action='ignore')
                                             .apply(pd.Series)
                                             .rename(mapper={0: 'latitude', 1: 'longitude'}, axis=1))
            df = df.drop(labels=['Unit ID', 'Ship position'], axis=1)
    elif wpk_id == '4':
        fname = topdir / f'AR{date:%y%m%d}.00{wpk_id}'
        if fname.exists():
            df = pd.read_csv(fname, skiprows=1, sep='\t', parse_dates=[['date', 'time']], index_col='date_time',
                             date_parser=lambda x: datetime.strptime(x, '%y/%m/%d %H:%M:%S'))        
    
    if len(df) > 0:
        # Convert wind speed and direction to u and v components
        df = add_wind_components(df)
        # Interpolate to minute time intervals
        df = interp_dataframe_time(df, date)
    return df

In [ ]:
def read_wpk_hourly(topdir, date, wpk_id):
    def date_parser(s):
        return datetime.strptime(s[:-4], '%Y-%m-%d %H:%M:%S')
        
    wpk_id = str(wpk_id)
    assert wpk_id in ['3', '4'], 'Works only for WeatherPacks  No. 3 or 4'
    
    # Read (raw?) data stored in hourly files and concatenate into a DataFrame for the whole day
    df = pd.DataFrame()
    for h in range(24):
        fname = topdir / wpk_id / f'{date:%Y}' / f'{date:%m}' / f'{date:%d}' / f'data_{wpk_id}_{date:%Y%m%d}_{h:02d}.log'
        time_col_name = ' zeno_date zeno_time zeno_timezone'
        if fname.exists():
            df_next = pd.read_csv(fname,
                                  error_bad_lines=False, warn_bad_lines=False,
                                  index_col=time_col_name,
                                  parse_dates=[time_col_name],
                                  date_parser=date_parser)
            df = pd.concat([df, df_next])

    if len(df) > 0:
        # Convert wind speed and direction to u and v components
        df = add_wind_components(df)
        # Interpolate to minute time intervals
        df = interp_dataframe_time(df, date)
    return df

In [ ]:
wpk_usage = wpk_usage.applymap(lambda x: 't,rh,ws,wd,p,sr,u,v')

In [ ]:
# date = pd.datetime(2018, 2, 6)

In [ ]:
df_full = pd.DataFrame()

for date in tqdm(wpk_usage.index):
    wpk_vars = wpk_usage.loc[date]
    data = dict()

    if wpk_vars.wpk2:
        vrbls = wpk_vars.wpk2.split(',')
        topdir = mypaths.wpk_dir / 'WP02'
        df = read_wpk_daily(topdir, date, '2')

        for vrbl in vrbls:
            for alias in vrbl_aliases[vrbl]:
                try:
                    data[vrbl+'_wpk2'] = df[alias]
                except KeyError:
                    pass

    if wpk_vars.wpk3:
        df = read_wpk_hourly(mypaths.wpk_dir, date, '3')
        vrbls = wpk_vars.wpk3.split(',')
        if len(df) > 0:
            for vrbl in vrbls:
                for alias in vrbl_aliases[vrbl]:
                    try:
                        data[vrbl+'_wpk3'] = df[alias]
                    except KeyError:
                        pass

    if wpk_vars.wpk4:
        vrbls = wpk_vars.wpk4.split(',')
        if date < datetime(2018, 2, 27):
            df = read_wpk_hourly(mypaths.wpk_dir, date, '4')
        else:
            df = read_wpk_daily(mypaths.wpk_dir / '2_Leg' / 'FORESTAR', date, '4')
        if len(df) > 0:
            for vrbl in vrbls:
                for alias in vrbl_aliases[vrbl]:
                    try:
                        data[vrbl+'_wpk4'] = df[alias]
                    except KeyError:
                        pass
                    
    df_full = pd.concat([df_full, pd.DataFrame(data)], sort=True)
# df_full.interpolate(method='time', inplace=True)

In [ ]:
df_flag = pd.read_csv('weatherpack_data_flag.csv', index_col='time', parse_dates=['time'])

In [ ]:
ds_full = df_full.to_xarray()
ds_flag = df_flag.to_xarray()

In [ ]:
ctd_temp = (xr.open_dataset(mypaths.igp_data_dir / 'ALL0118_uctd' / 'igp_all0118_uctd.nc')
            .sbe38_bow_temperature
            .to_dataset())

In [ ]:
freqs_min = [1, 10, 60]

In [ ]:
with open('weatherpack_notes.txt', 'r') as f:
    wpk_notes = f.read()

In [ ]:
global_attrs = {
    'title': 'NMEA, WeatherPack, and CTD Temperature data from IGP cruise (QC level 1)',
    'summary': """The dataset comprises data gathered during the IGP field campaign by several instruments on board NRV Alliance.
The data are aligned in time: i.e. CTD and NMEA data (having originally 1 sec resolution) are averaged over 1 min intervals; Weatherpack data typically have 1 min resolution, but sometimes the timestamp is between two whole minute marks, so the data is interpolated to 1 min resolution for all days.
There are large gaps in useful data from WeatherPacks due to incorrect set-up of Wpk 3 and 4 during cruise leg 1 and occasional freezing of sensors.
Obviously incorrect values are flagged: e.g. <variable>_wpk<wpk_number>_flag = 1.""",
    'keywords': 'igp,weatherpack,aws,nmea,ctd',
    'conventions': 'CF-1.7,ACDD-1.3',
    'history': f'Combined from different sources as the netCDF file at {datetime.utcnow():%Y-%m-%d %H:%M:%S} GMT by Denis Sergeev (University of East Anglia)',
    'source': 'NMEA on board NRV Alliance; WeatherPacks 2, 3, 4; SBE38 temperature sensor',
    'processing_level': 'no calibration performed',
    'comment':  wpk_notes,
    'date_created': f'{datetime.utcnow():%Y-%m-%d %H:%M:%S} GMT',
    'institution': 'University of East Anglia, Woods Hole Oceanographic Institution',
    'creator_name': 'Denis Sergeev',
    'creator_email': 'd.sergeev@uea.ac.uk',
    'creator_institution': 'University of East Anglia',
    'project': 'The Iceland Greenland Seas Project (IGP)',
    'coverage_content_type': 'physicalMeasurement',
}

In [ ]:
outdir = mypaths.igp_data_dir / 'wpk_nmea_ctd' / 'qc1'
outdir.mkdir(parents=True, exist_ok=True)

In [ ]:
fillval = 1e20

In [ ]:
for date in tqdm(wpk_usage.index[40:]):
    ds_dict = dict()

    # NMEA data
    fname = mypaths.igp_data_dir / 'nmea_logs' / f'{date:%Y%m%d}.log'
    AC = AllianceComposite(fname, date)
    try:
        AC.process(MSG_LIST)
        ds_nmea = AC.ds.rename({k: k + '_nmea' for k in AC.ds.data_vars})
        ds_dict['nmea'] = ds_nmea
    except FileNotFoundError:
        pass
#     combined_ds.merge(ds_nmea, inplace=True)
    # CTD data
    try:
        ds_dict['ctd'] = ctd_temp.sel(time=f'{date:%Y%m%d}')
    except KeyError:
        pass
#     combined_ds = combined_ds.assign(sbe38_bow_temperature=ds_ctd.sbe38_bow_temperature)
    # weatherpacks data
    ds = ds_full.sel(time=f'{date:%Y%m%d}').merge(ds_flag.sel(time=f'{date:%Y%m%d}'))
    for data_var in filter(lambda x: '_flag' not in x, ds.data_vars):
        wpk_num = data_var[-1]
        for vrbl, vdict in vrbl_attrs.items():
            if vrbl == data_var.split('_wpk')[0]:
                attrs = vdict['attrs'].copy()
                attrs['long_name'] += f' from WeatherPack {wpk_num}'
                scl = vdict.get('scl', 1)
                flag_var = vdict.get('dep', data_var)
        if isinstance(flag_var, list):
            mask = np.zeros_like(ds[data_var], dtype=bool)
            for fv in flag_var:
                mask = np.bitwise_or(mask, (ds[fv+data_var[-5:]]==1))
            ds.assign(**{data_var+'_flag': mask})
        else:
            mask = (ds[data_var+'_flag'] == 1)

        ds[data_var][mask] = np.nan
        ds[data_var] *= scl
        ds[data_var].attrs.update(attrs)
        
    ds_dict['wpk'] = ds[[i for i in ds.data_vars
                       if i[:2] not in ['wd', 'ws'] and not i.startswith('sr_wpk2')]]
    

        
    for freq in tqdm(freqs_min):
        pd_freq = pd.Timedelta(freq, 'm')  # in minutes
        combined_ds = xr.Dataset(attrs=global_attrs)
        for k, v in ds_dict.items():
#             if (v.time[1] - v.time[0]).values < pd_freq.to_timedelta64():
            if v.time.shape[0] > 0:
                ds = (v.resample(time=pd_freq, keep_attrs=True)
                      .reduce(np.nanmean, keep_attrs=True))
                if k == 'wpk':
                    ds = ds.drop(labels=[i for i in ds.data_vars
                                         if i.endswith('_flag')])
    #             else:
    #                 ds = v
                combined_ds.merge(ds, inplace=True)
        date_attrs = {
            'time_coverage_start': f'{np.nanmin(combined_ds.time).astype("datetime64[us]").astype(datetime):%Y-%m-%dT%H:%M:%SZ}',
            'time_coverage_end': f'{np.nanmax(combined_ds.time).astype("datetime64[us]").astype(datetime):%Y-%m-%dT%H:%M:%SZ}',
        }
        date_attrs['time_coverage_resolution'] = f'{freq}min'
        combined_ds.attrs.update(date_attrs)
        combined_ds = combined_ds.fillna(fillval)
        for data_var in combined_ds.data_vars:
            combined_ds[data_var].attrs.update(_FillValue=fillval)

        combined_ds.to_netcdf(path=outdir / f'wpk_nmea_ctd_qc1_{date:%Y%m%d}_{freq:02d}min.nc',
                              encoding=dict(time=dict(units=f'seconds since {AC.TSTART:%Y-%m-%dT%H:%M:%S.%f}',
                                            calendar='gregorian')))